In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize, to_rgb
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import torch.nn.functional as F
import random
from skimage import measure, color
from matplotlib.cm import ScalarMappable
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle
import matplotlib.colors as mcolors

In [ ]:
# Model definition
number_raster_layers = 9
number_pixels_layer = 19

class CNNRegressor(nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.conv1 = nn.Conv2d(number_raster_layers, 16, kernel_size=3, stride=1, padding=1)
        self.selu1 = nn.SELU()
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.selu2 = nn.SELU()
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.selu3 = nn.SELU()
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.selu4 = nn.SELU()
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.selu5 = nn.SELU()
        self.conv6 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.selu6 = nn.SELU()
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(512 * number_pixels_layer * number_pixels_layer, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.selu1(x)
        x = self.conv2(x)
        x = self.selu2(x)
        x = self.conv3(x)
        x = self.selu3(x)
        x = self.conv4(x)
        x = self.selu4(x)
        x = self.conv5(x)
        x = self.selu5(x)
        x = self.conv6(x)
        x = self.selu6(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

model = CNNRegressor()

In [ ]:
# Load model
path_model = '../../Data/Calibrated_models/global_regressor_V0.pth'
model.load_state_dict(torch.load(path_model))


In [ ]:
# Cambia el modelo al modo de evaluación (si es necesario)
model.eval()

In [ ]:
# Load tensors
path_tensor_train = '../../Data/Calibrated_models/global_regressor_V0_tensor_y_train.pth'
y_train = torch.load(path_tensor_train)

path_tensor_test = '../../Data/Calibrated_models/global_regressor_V0_tensor_y_test.pth'
y_test = torch.load(path_tensor_test)

path_tensor_test = '../../Data/Calibrated_models/global_regressor_V0_test_tensor.pth'
test_tensor = torch.load(path_tensor_test)

path_tensor_training = '../../Data/Calibrated_models/global_regressor_V0_training_tensor.pth'
training_tensor = torch.load(path_tensor_training)

In [ ]:
import pandas as pd
# DataFrame to store all the results for pixel importance
result_df = pd.DataFrame()

In [ ]:
# Iterate over all elements of the first dimension of test_tensor
for input_tensor_number in range(test_tensor.shape[0]):
   
    print(input_tensor_number)

    input_tensor_raw = test_tensor[input_tensor_number].unsqueeze(0)  # Step 9

    # Convert the input tensor to float to manipulate it with the CNN
    input_tensor = input_tensor_raw.float()
    
    input_tensor_squeeze = input_tensor.squeeze(0)

    # Define the color palette according to land cover types
    color_map = {
        0: 'saddlebrown',   # Bare
        1: 'red',           # BuiltUp
        2: 'yellow',        # Crops
        3: 'lightgreen',    # Grass
        4: 'limegreen',     # MossLichen
        5: 'blue',          # PermanentWater
        6: 'cyan',          # SeasonalWater
        7: 'olive',         # Shrub
        8: 'darkgreen'      # Tree
    }

    # Determine the dominant layer for each pixel
    dominant_layer_indices = np.argmax(input_tensor_squeeze, axis=0)

    # Create an empty RGB image
    dominant_image = np.zeros((*dominant_layer_indices.shape, 3), dtype=np.uint8)

    # Map each index to its corresponding color in the RGB image
    for index, color in color_map.items():
        mask = dominant_layer_indices == index
        dominant_image[mask] = np.array(to_rgb(color)) * 255  # Convert color to RGB and adjust to scale 0-255

    # Define the color palette according to land cover types
    color_map = {
        0: 'saddlebrown',   # Bare
        1: 'red',           # BuiltUp
        2: 'yellow',        # Crops
        3: 'lightgreen',    # Grass
        4: 'limegreen',     # MossLichen
        5: 'blue',          # PermanentWater
        6: 'cyan',          # SeasonalWater
        7: 'olive',         # Shrub
        8: 'darkgreen'      # Tree
    }

    # Labels for the legend based on category names
    labels = {
        0: "Bare",
        1: "Built-Up",
        2: "Crops",
        3: "Grass",
        4: "Moss/Lichen",
        5: "Permanent Water",
        6: "Seasonal Water",
        7: "Shrub",
        8: "Tree"
    }

    # Plot image (sanity check)
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.imshow(dominant_image)
    ax.axis('on')

    # Create a list of patches for the legend
    patches = [mpatches.Patch(color=to_rgb(color), label=labels[idx]) for idx, color in color_map.items()]

    # Add the legend to the plot
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    ax.xaxis.set_ticks([]) 
    ax.yaxis.set_ticks([]) 

    # Show the plot
    plt.show()

    ##################################################
    # CREATE GRAD-CAM HEATMAP
    ##################################################
    
    class GradCAM:
        def __init__(self, model, layer):
            self.model = model
            self.layer = layer
            self.gradient = None
            self.activation = None

            self.hook_handles = []
            self.hook_handles.append(layer.register_forward_hook(self.save_activation))
            self.hook_handles.append(layer.register_backward_hook(self.save_gradient))

        def save_activation(self, module, input, output):
            self.activation = output.detach()

        def save_gradient(self, module, grad_input, grad_output):
            self.gradient = grad_output[0].detach()

        def __call__(self, x, index=None):
            # Set a fixed seed for reproducibility
            if seed is not None:
                torch.manual_seed(seed)
                torch.cuda.manual_seed_all(seed)
                np.random.seed(seed)
                random.seed(seed)

            # Clear previous gradients and activations
            self.gradient = None
            self.activation = None

            output = self.model(x)
            if index is None:
                index = torch.argmax(output)

            self.model.zero_grad()
            output.backward(torch.ones_like(output), retain_graph=True)

            pooled_gradients = torch.mean(self.gradient, dim=[0, 2, 3])
            for i in range(pooled_gradients.size(0)):
                self.activation[:, i, :, :] *= pooled_gradients[i]

            heatmap = torch.mean(self.activation, dim=1).squeeze()
            heatmap = F.relu(heatmap)
            heatmap /= torch.max(heatmap)

            return heatmap

        def release(self):
            for handle in self.hook_handles:
                handle.remove()

    ###############################################################
    ###############################################################

    # Use GradCAM on the model with the conv6 layer
    grad_cam = GradCAM(model, model.conv6)
    # Set a fixed seed to ensure reproducibility
    seed = 42
    # Obtain the heatmap for grad-CAM
    heatmap = grad_cam(input_tensor)
    grad_cam.release()

    # Ensure the heatmap is normalized
    normalized_heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())

    # Create an RGBA image for `dominant_image` where the alpha channel is adjusted according to the heatmap
    dominant_rgba_image = np.zeros((*dominant_image.shape[:2], 4), dtype=np.float32) 

    for i in range(3):  # Copy the RGB channels
        dominant_rgba_image[..., i] = dominant_image[..., i] / 255.0  # Normalize and copy
    dominant_rgba_image[..., 3] = normalized_heatmap  # Adjust the alpha channel using the heatmap

    # Plot image (sanity check)
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.imshow(dominant_rgba_image)  # Show the RGBA image
    ax.axis('on')  # Show the axes

    # Define the color palette and labels for the `dominant_image` legend
    color_map = {
        "Bare": 'saddlebrown',
        "BuiltUp": 'red',
        "Crops": 'yellow',
        "Grass": 'lightgreen',
        "MossLichen": 'limegreen',
        "PermanentWater": 'blue',
        "SeasonalWater": 'cyan',
        "Shrub": 'olive',
        "Tree": 'darkgreen'
    }
    patches = [mpatches.Patch(color=to_rgb(color), label=label) for label, color in color_map.items()]

    # Hide axis labels
    ax.xaxis.set_ticks([])  # Hide x-axis labels and ticks
    ax.yaxis.set_ticks([])  # Hide y-axis labels and ticks

    # Add the legend for land cover
    legend = ax.legend(handles=patches, loc='upper left', title="Land Cover Types", bbox_to_anchor=(1.05, 1), borderaxespad=0.)

    plt.show()

    # Assuming 'dominant_image' and 'heatmap' are defined
    # Ensure the heatmap is normalized
    normalized_heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())

    # Create an RGBA image for `dominant_image` where the alpha channel is adjusted according to the heatmap
    dominant_rgba_image = np.zeros((*dominant_image.shape[:2], 4), dtype=np.float32)  # Create RGBA image

    for i in range(3):  # Copy the RGB channels
        dominant_rgba_image[..., i] = np.around(dominant_image[..., i] / 255.0, 7)  # Normalize and copy

    ########################################################
    # Adjust the alpha channel using the heatmap with the threshold
    threshold = 0.0
    #######################################################

    alpha_channel = np.where(normalized_heatmap > threshold, 1.0, 0)
    dominant_rgba_image[..., 3] = alpha_channel  # Apply the modified alpha channel

    # Create image
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.imshow(dominant_rgba_image)
    ax.axis('on')

    # Define the color palette and labels for the `dominant_image` legend
    color_map = {
        "Bare": 'saddlebrown',
        "BuiltUp": 'red',
        "Crops": 'yellow',
        "Grass": 'lightgreen',
        "MossLichen": 'limegreen',
        "PermanentWater": 'blue',
        "SeasonalWater": 'cyan',
        "Shrub": 'olive',
        "Tree": 'darkgreen'
    }
    patches = [mpatches.Patch(color=to_rgb(color), label=label) for label, color in color_map.items()]
    ax.xaxis.set_ticks([]) 
    ax.yaxis.set_ticks([])

    # Add the legend for land cover
    legend = ax.legend(handles=patches, loc='upper left', title="Land Cover Types", bbox_to_anchor=(1.05, 1), borderaxespad=0.)

    plt.show()

    # Sanity check

    # Assuming `dominant_rgba_image` is your original RGBA image
    alpha_mask = dominant_rgba_image[..., 3] == 1

    # Create an output image that will be a copy of the original image
    output_image = np.zeros_like(dominant_rgba_image)

    # Copy only the pixels where alpha is 1
    output_image[alpha_mask] = dominant_rgba_image[alpha_mask]

    # Visualize the resulting image, which will now highlight the pixels with alpha = 1
    plt.figure(figsize=(10, 6))
    plt.imshow(output_image)
    plt.title("Pixels with Alpha = 1")
    plt.axis('on')
    plt.show()

    #############################################
    # EXTRACT CLUSTERS
    #############################################
    
    # Convert the RGB image (ignoring alpha) to a binary image where there are non-black pixels
    binary_image = np.any(output_image[..., :3] > 0, axis=-1)  # True where there is color

    # Apply connected component labeling on the binary image
    labeled_image = measure.label(binary_image, connectivity=2)  # Diagonal connections included

    # Ensure that output_image only contains RGB channels
    if output_image.shape[-1] == 4:  # If it still includes the alpha channel
        output_image = output_image[..., :3]  # Take only the first three channels (RGB)

    try:
        image_labeled = color.label2rgb(labeled_image, output_image, kind='overlay', bg_label=0)
        plt.figure(figsize=(10, 6))
        plt.imshow(image_labeled)
        plt.title("Connected components")
        plt.axis('off')
        plt.show()
    except Exception as e:
        print("Error converting labels to RGB:", e)

    #############################################
    # SAVE INFO AS A DATAFRAME
    #############################################

    # Prepare data for the DataFrame
    data = {
        'x': [],
        'y': [],
        'cluster_label': [],
        'grad_cam_value': []
    }

    # Iterate over each pixel in the labeled image
    for (x, y), label in np.ndenumerate(labeled_image):
        data['x'].append(x)
        data['y'].append(y)
        data['cluster_label'].append(label)
        # Extract the value from the tensor using .item() and add it to the DataFrame
        data['grad_cam_value'].append(heatmap[x, y].item())

    # Create the DataFrame
    df = pd.DataFrame(data)

    # Display the DataFrame
    print(df)

    ##############################
    # Extract pixel's land cover from dominant image

    # Mapping of color names to RGB (example values, adjust as necessary)
    rgb_color_map = {
        'saddlebrown': [139, 69, 19],
        'red': [255, 0, 0],
        'yellow': [255, 255, 0],
        'lightgreen': [144, 238, 144],
        'limegreen': [50, 205, 50],
        'blue': [0, 0, 255],
        'cyan': [0, 255, 255],
        'olive': [128, 128, 0],
        'darkgreen': [0, 100, 0]
    }

    # Reverse the color_map to get a mapping from RGB to ID
    rgb_to_id = {tuple(value): key for key, value in rgb_color_map.items()}

    id_to_color = {
        0: 'saddlebrown',   # Bare
        1: 'red',           # BuiltUp
        2: 'yellow',        # Crops
        3: 'lightgreen',    # Grass
        4: 'limegreen',     # MossLichen
        5: 'blue',          # PermanentWater
        6: 'cyan',          # SeasonalWater
        7: 'olive',         # Shrub
        8: 'darkgreen'      # Tree
    }

    def map_pixels_to_ids(image):
        # Assign a default ID for unmapped pixels
        id_image = np.full((image.shape[0], image.shape[1]), -1, dtype=int)

        # Convert the image to a list of tuples for efficient lookup
        flattened_image = [tuple(pixel) for pixel in image.reshape(-1, image.shape[2])]

        # Map each pixel using the dictionary, with a default ID if not found
        id_flat = [rgb_to_id.get(pixel, -1) for pixel in flattened_image]

        # Reconstruct the ID image
        id_image = np.array(id_flat).reshape(image.shape[0], image.shape[1])

        return id_image

    # Apply the function to the image
    color_matrix = map_pixels_to_ids(dominant_image)

    # Direct mapping from ID to color name
    id_to_color = {
        0: 'saddlebrown',   # Bare
        1: 'red',           # BuiltUp
        2: 'yellow',        # Crops
        3: 'lightgreen',    # Grass
        4: 'limegreen',     # MossLichen
        5: 'blue',          # PermanentWater
        6: 'cyan',          # SeasonalWater
        7: 'olive',         # Shrub
        8: 'darkgreen'      # Tree
    }

    # Labels for each ID
    labels = {
        0: "Bare",
        1: "Built-Up",
        2: "Crops",
        3: "Grass",
        4: "Moss/Lichen",
        5: "Permanent Water",
        6: "Seasonal Water",
        7: "Shrub",
        8: "Tree"
    }

    # Create a dictionary to map from color to land cover label
    color_to_label = {color: labels[id] for id, color in id_to_color.items()}
    # Convert color names to land cover labels
    label_matrix = [[color_to_label.get(color, 'Unknown') for color in row] for row in color_matrix]

    # Create a list to store the dictionaries
    data_landcover = []

    # Iterate over each row and each column in the matrix
    for y, row in enumerate(label_matrix):
        for x, label in enumerate(row):
            # Create a dictionary for each cell and add it to the list
            data_landcover.append({
                'x': x,
                'y': y,
                'land_cover': label,
                'site': input_tensor_number
            })

    # Create a DataFrame from the list of dictionaries
    df_landcover = pd.DataFrame(data_landcover)

    # Display the DataFrame
    print(df_landcover)

    # Assuming df and df_landcover are already defined as in your description
    # Perform the left join
    df_merged = pd.merge(df_landcover, df, on=['x', 'y'], how='left')

    # Add the DataFrame to the total results DataFrame
    result_df = pd.concat([result_df, df_merged], ignore_index=True)


In [ ]:
# Save the resulting DataFrame as a CSV file
result_df.to_csv('../../Data/df_features_visualization.csv', index=False)